# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/callistaswart/Projects/python-api-challenge/Starter_Code/output_data/cities.csv")


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display


# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color= "City"
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values

preferred_cities_df = city_data_df.loc[(city_data_df["Wind Speed"] <= 10) & (city_data_df["Cloudiness"] <= 10) & \
                     (city_data_df["Max Temp"] >= 60) & (city_data_df["Max Temp"] <= 80) & (city_data_df["Humidity"] < 60)].dropna()


# Drop any rows with null values
preferred_cities_df.dropna

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
19,19,puerto ayora,-0.7393,-90.3518,22.98,82,41,1.79,EC,1666108242
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220
27,27,bambous virieux,-20.3428,57.7575,22.14,68,75,3.60,MU,1666108246
38,38,labuhan,-6.8844,112.2051,25.24,88,100,2.74,ID,1666108254
39,39,tomatlan,19.9333,-105.2500,28.05,71,93,0.58,MX,1666108255
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
46,46,mogadishu,2.0371,45.3438,25.67,83,13,6.45,SO,1666108258
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=preferred_cities_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
9,saint-pierre,RE,-21.3393,55.4781,68,
19,puerto ayora,EC,-0.7393,-90.3518,82,
20,avarua,CK,-21.2078,-159.7750,73,
27,bambous virieux,MU,-20.3428,57.7575,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel

params = {
     "catagories":"accommodation.hotel",
     "limit":20,
     "apiKey":geoapify_key,
     "radius":100000
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude="Lat"
    longitude="Lng"
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
     #print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
9,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
19,puerto ayora,EC,-0.7393,-90.3518,82,No hotel found
20,avarua,CK,-21.2078,-159.7750,73,No hotel found
27,bambous virieux,MU,-20.3428,57.7575,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

map = hotel_df.hvplot.points( 
    "Lng",
    "Lat",
    geo = True,
    color = "City",
    size = "Humidity",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    hover_cols = ["Hotel Name","Country"]
)


# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)